In [ ]:
import syft as sy
from syft.service.queue.queue_stash import QueueItem
from syft.service.queue.zmq_queue import ZMQProducer
from syft.client.api import SyftAPICall
from syft.service.response import SyftNotReady
import time

In [ ]:
domain = sy.Worker.named(name='Domain', processes=1, reset=True)
domain

In [ ]:
client = domain.root_client
client

In [ ]:
client.api

In [ ]:
assert domain.queue_manager is not None

In [ ]:
assert len(domain.queue_manager.producers) > 0

In [ ]:
assert len(domain.queue_manager.consumers) > 0

In [ ]:
call = SyftAPICall(
    node_uid=domain.id, path="data_subject.get_all", args=(), blocking=False, kwargs={}
)
call

In [ ]:
signed_api_call = call.sign(client.credentials)
signed_api_call

In [ ]:
assert signed_api_call.credentials == client.credentials.verify_key

In [ ]:
signed_result = domain.handle_api_call(signed_api_call)
signed_result

In [ ]:
assert signed_result.credentials == client.credentials.verify_key

In [ ]:
queue_item = signed_result.message.data
queue_item

In [ ]:
assert queue_item is not None
assert isinstance(queue_item, QueueItem)

In [ ]:
queue_item.syft_client_verify_key

In [ ]:
MAX_RETRIES = 5
count = 0
while(not signed_result.message.data.resolved):
    # poll database
    queue_item.resolve
    count += 1
    time.sleep(1)
    if count == MAX_RETRIES:
        break

In [ ]:
queue_item

In [ ]:
queue_item.resolve

In [ ]:
queue_item

In [ ]:
list(domain.document_store.partitions["QueueItem"].data.values())

In [ ]:
assert len(domain.document_store.partitions["QueueItem"].data) == 0

In [ ]:
assert queue_item.resolved is True

In [ ]:
queue_item.result

In [ ]:
assert queue_item.result is not None

In [ ]:
assert queue_item.resolve.data == []

In [ ]:
domain.queue_manager._client.purge_all()

In [ ]:
assert domain.queue_manager._client.producers['api_call'].alive
assert domain.queue_manager._client.consumers['api_call'][0].alive

In [ ]:
domain.queue_manager.close()